# This notebook tutorial details generation simulation data with prosstt

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import torch
import newick
import numpy as np
import pandas as pd
import scanpy as sc
from numpy import random

from prosstt import tree
from prosstt import simulation as sim
from prosstt import sim_utils as sut

from sklearn.preprocessing import MinMaxScaler

import seaborn as sns
import matplotlib.pyplot as plt

rseed = 0
random.seed(rseed)

In [2]:
import os
import sys

parent_dir = os.path.dirname('/home/jupyter/')
sys.path.append(parent_dir)

In [3]:
newick_string = "(B:2000,C:2000,D:2000,E:2000)A:2000;"
t = tree.Tree.from_newick(newick_string, genes=1500, modules=100)
sample_time = np.arange(0, t.get_max_time())

In [4]:
uMs, Ws, Hs = sim.simulate_lineage(t, a=0.05)
gene_scale = sut.simulate_base_gene_exp(t, uMs)
t.add_genes(uMs, gene_scale)

In [5]:
alpha = np.exp(random.normal(loc=np.log(0.2), scale=np.log(1.5), size=t.G)) # used for final simulation
beta = np.exp(random.normal(loc=np.log(1), scale=np.log(1.5), size=t.G)) + 1 # used for final simulation
X, pseudotimes, branch, scalings = sim.sample_whole_tree(t, 1, alpha=alpha, beta=beta) # used for final simulation

In [6]:
X1 = (X.transpose() / scalings).transpose()
adata = sc.AnnData(np.log(X1+1))

sc.pp.neighbors(adata, use_rep="X")
sc.tl.diffmap(adata)
dm1 = adata.obsm["X_diffmap"]
adata.obsm["X_umap"] = adata.obsm["X_diffmap"][:,1:3]
br_names1, indices1 = np.unique(branch, return_inverse=True)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [7]:
adata.obs['Branch']=branch
adata.obs['Pseudotime']=pseudotimes

In [ ]:
# We did some visualization of marker genes in each cell state
sc.tl.rank_genes_groups(adata, 'Branch', method='wilcoxon')
markers = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(20)
ax = sc.pl.heatmap(adata, markers.T.values.flatten().tolist(), groupby='Branch', cmap='RdBu_r')

### Cell state A will be used as basal state

In [13]:
Xc = X[adata.obs['Branch']=='A',:]
control = adata[adata.obs['Branch']=='A',:]

In [14]:
# We rescale pseudotime from 0 to 1
ptime = control.obs['Pseudotime']
scaler = MinMaxScaler(feature_range=(0, 1))
ptime = scaler.fit_transform(np.expand_dims(ptime, 1))
pseudo = torch.Tensor(ptime).view(-1,1)

In [16]:
C = Xc.sum(1).mean()
# print(C)

In [17]:
p_cont = Xc
q_cont = p_cont.sum(0)
P_cont = q_cont/q_cont.sum()
P_cont = torch.Tensor(P_cont).view(1,-1)
gt_control = P_cont.cpu().numpy().squeeze()

poisson = torch.distributions.Poisson((C+1) * torch.matmul(torch.ones(Xc.shape[0],1),P_cont))
cont = Xc + np.asarray(poisson.sample().detach().cpu())

### Simulation 1

In [19]:
p1 = X[adata.obs['Branch']=='B',:]
q = p1.sum(0)
P = q/q.sum()
P = torch.Tensor(P).view(1,-1)
gt_p1 = P.cpu().numpy().squeeze()

poisson_pert = torch.distributions.Poisson(C * torch.matmul(pseudo,P))
poisson_cont = torch.distributions.Poisson(C * torch.matmul((1-pseudo),P_cont))

pert1_basal = Xc + np.asarray(poisson_cont.sample().detach().cpu()).copy()
pert1_pert = np.asarray(poisson_pert.sample().detach().cpu()).copy()

pert1 = pert1_basal + pert1_pert

In [20]:
p2 = X[adata.obs['Branch']=='E',:]
q = p2.sum(0)
P = q/q.sum()
P = torch.Tensor(P).view(1,-1)
gt_p2 = P.cpu().numpy().squeeze()

poisson_pert = torch.distributions.Poisson(C * torch.matmul(pseudo,P))
poisson_cont = torch.distributions.Poisson(C * torch.matmul((1-pseudo),P_cont))

pert2_basal = Xc + np.asarray(poisson_cont.sample().detach().cpu()).copy()
pert2_pert = np.asarray(poisson_pert.sample().detach().cpu()).copy()

pert2 = pert2_basal + pert2_pert 

### Simulation 2

In [67]:
p1 = X[adata.obs['Branch']=='D',:]
q = p1.sum(0)
P = q/q.sum()
P = torch.Tensor(P).view(1,-1)
gt_shared = P.cpu().numpy().squeeze()

poisson_pert = torch.distributions.Poisson(C * torch.matmul(pseudo,P))
poisson_cont = torch.distributions.Poisson(C * torch.matmul((1-pseudo),P_cont))

pert1_basal = Xc + np.asarray(poisson_cont.sample().detach().cpu()).copy()
pert1_pert = np.asarray(poisson_pert.sample().detach().cpu()).copy()
pert1 = pert1_basal + pert1_pert

In [70]:
p2 = X[adata.obs['Branch']=='D',:]
q = p2.sum(0)
P = q/q.sum()
P = torch.Tensor(P).view(1,-1)

poisson_pert = torch.distributions.Poisson(C * torch.matmul(pseudo,P) * 0.5)
poisson_cont = torch.distributions.Poisson(C * torch.matmul((1-pseudo),P_cont))

pert2_basal = Xc + np.asarray(poisson_cont.sample().detach().cpu()).copy()
pert2_pert = np.asarray(poisson_pert.sample().detach().cpu()).copy()

pert2 = pert2_basal + pert2_pert

### Simulation 3

In [28]:
p_shared = X[adata.obs['Branch']=='D',:]
q_shared = p_shared.sum(0)
P_shared = q_shared/q_shared.sum()
P_shared = torch.Tensor(P_shared).view(1,-1)
gt_shared = P_shared.cpu().numpy().squeeze() * C

poisson_shared = torch.distributions.Poisson(C * torch.matmul((1-pseudo),P_shared))

In [29]:
p1 = X[adata.obs['Branch']=='B',:]
q = p1.sum(0)
P = q/q.sum()
P = torch.Tensor(P).view(1,-1)
gt_p1 = P.cpu().numpy().squeeze() * C

poisson_pert = torch.distributions.Poisson(C * torch.matmul(pseudo,P))

pert1_basal = Xc
pert1_pert = np.asarray(poisson_pert.sample().detach().cpu()).copy()
pert1_share = np.asarray(poisson_shared.sample().detach().cpu())
pert1 = pert1_basal + pert1_pert + pert1_share

In [30]:
p2 = X[adata.obs['Branch']=='E',:]
q = p2.sum(0)
P = q/q.sum()
P = torch.Tensor(P).view(1,-1)
gt_p2 = P.cpu().numpy().squeeze() * C

poisson_pert = torch.distributions.Poisson(C * torch.matmul(pseudo,P))

pert2_basal = Xc
pert2_pert = np.asarray(poisson_pert.sample().detach().cpu()).copy()
pert2_share = np.asarray(poisson_shared.sample().detach().cpu())
pert2 = pert2_basal + pert2_pert + pert2_share

### Run code of simulation case above individually and create anndata below

In [21]:
control = sc.AnnData(X=cont)
control.obs['State']='A'
control.obs['Pseudotime']=ptime
control.obs['Condition']="Control"

control.layers['Basal']=control.X
control.layers['Pert']=np.zeros_like(control.X) # control.X #
control.layers['Share']=np.zeros_like(control.X) # control.X #

In [22]:
perturb1 = sc.AnnData(X=pert1)
perturb1.obs['State']='A'
perturb1.obs['Pseudotime']=ptime
perturb1.obs['Condition']="Perturbation1"

perturb1.layers['Basal']=pert1_basal
perturb1.layers['Pert']=pert1_pert
# perturb1.layers['Share']=pert1_share

In [23]:
perturb2 = sc.AnnData(X=pert2)
perturb2.obs['State']='A'
perturb2.obs['Pseudotime']=ptime
perturb2.obs['Condition']="Perturbation2"


perturb2.layers['Basal']=pert2_basal
perturb2.layers['Pert']=pert2_pert
# perturb2.layers['Share']=pert2_share

In [24]:
sim = control.concatenate(perturb1,perturb2)
sim.write("../data/simulation_data.h5ad")